In [5]:
import cv2
import numpy as np

def detect_bends(image_path, threshold=100, min_line_length=50, max_line_gap=10):
    """Detects bends in an image, even when overlapping.

    Args:
        image_path: Path to the input image.
        threshold: Hough Line Transform threshold (adjust for bend sensitivity).
        min_line_length: Minimum line length to be considered a bend.
        max_line_gap: Maximum gap between line segments to be considered a single bend.

    Returns:
        List of bend coordinates (x1, y1, x2, y2) and an annotated image.
    """

    # 1. Load and Preprocess
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Reduce noise
    edges = cv2.Canny(blurred, 50, 150)

    # 2. Hough Line Transform (Find Lines)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, min_line_length, max_line_gap)

    # 3. Cluster Overlapping Lines (Key Improvement)
    bend_clusters = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        clustered = False
        for cluster in bend_clusters:
            for line_in_cluster in cluster:
                # Check if the line is close to any line in an existing cluster
                if (
                    abs(x1 - line_in_cluster[0]) < 20
                    and abs(y1 - line_in_cluster[1]) < 20
                    or abs(x2 - line_in_cluster[2]) < 20
                    and abs(y2 - line_in_cluster[3]) < 20
                ):
                    cluster.append(line[0])  
                    clustered = True
                    break
        if not clustered:
            bend_clusters.append([line[0]]) 

    # 4. Visualize and Extract Coordinates
    bend_coords = []  # Store final bend coordinates
    for cluster in bend_clusters:
        bend_coords.append(cluster[0])  # Take representative line from each cluster
        x1, y1, x2, y2 = cluster[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    return bend_coords, img


# Example Usage
image_path = 'Shapes/row_16.png'  # Replace with your image path
bend_coords, annotated_image = detect_bends(image_path)
print(f"Detected bend coordinates: {bend_coords}")
cv2.imshow("Bends Detected", annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


TypeError: 'NoneType' object is not iterable

In [6]:
import cv2
import numpy as np

def detect_bends(image_path, threshold=100, min_line_length=50, max_line_gap=10):
    """Detects bends in an image, even when overlapping.

    Args:
        image_path: Path to the input image.
        threshold: Hough Line Transform threshold (adjust for bend sensitivity).
        min_line_length: Minimum line length to be considered a bend.
        max_line_gap: Maximum gap between line segments to be considered a single bend.

    Returns:
        List of bend coordinates (x1, y1, x2, y2) and an annotated image.
    """

    # 1. Load and Preprocess
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return [], None  # Return empty list and None for image if reading fails

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)  # Reduce noise
    edges = cv2.Canny(blurred, 50, 150)

    # 2. Hough Line Transform (Find Lines)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold, min_line_length, max_line_gap)

    # Handle the case where no lines are found (lines is None)
    if lines is None:
        print("No bends detected in the image.")
        return [], img  # Return empty list of bend coordinates and the image

    # 3. Cluster Overlapping Lines (Key Improvement)
    bend_clusters = []
    for line in lines:  # Now it's safe to iterate over lines
        x1, y1, x2, y2 = line[0]
        clustered = False
        for cluster in bend_clusters:
            for line_in_cluster in cluster:
                # Check if the line is close to any line in an existing cluster
                if (
                    abs(x1 - line_in_cluster[0]) < 20
                    and abs(y1 - line_in_cluster[1]) < 20
                    or abs(x2 - line_in_cluster[2]) < 20
                    and abs(y2 - line_in_cluster[3]) < 20
                ):
                    cluster.append(line[0])
                    clustered = True
                    break
        if not clustered:
            bend_clusters.append([line[0]])

    # 4. Visualize and Extract Coordinates
    bend_coords = []  # Store final bend coordinates
    for cluster in bend_clusters:
        bend_coords.append(cluster[0])  # Take representative line from each cluster
        x1, y1, x2, y2 = cluster[0]
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    return bend_coords, img


# Example Usage
image_path = 'Shapes/row_16.png'  # Replace with your image path
bend_coords, annotated_image = detect_bends(image_path)
print(f"Detected bend coordinates: {bend_coords}")

if annotated_image is not None:
    cv2.imshow("Bends Detected", annotated_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


No bends detected in the image.
Detected bend coordinates: []
